In [4]:


ROSACEA_MODEL = r"D:\Projects\Python Devolpment\Data field\python\data scintict\medicals-models\_models\rosacea.pt"
MOST_MEDICAL_MODELS = r"D:\Projects\Python Devolpment\Data field\python\data scintict\medicals-models\_models\4medical_X.pt"


def yolo_predection(img):
    # results = model.predict(source=img,save_crop=True,device='gpu')
    model = YOLO(MOST_MEDICAL_MODELS)

    results = model.predict(
        source=img, 
        save_crop=True,
        save_txt=True, 
        device="cpu", 
    )

    return results ,model


 

In [ ]:
results[0].boxes

In [5]:
from concurrent.futures import ThreadPoolExecutor
from ultralytics import YOLO

In [25]:

import numpy as np

import cv2
from concurrent.futures import ThreadPoolExecutor
import tensorflow as tf



MOST_MEDICAL_MODELS = r'D:\Projects\Python Devolpment\Data field\python\data scintict\medicals-models\_models\yolo_4_midecal.pt'
ACNE_MODEL = r"D:\Projects\Python Devolpment\Data field\python\data scintict\medicals-models\_models\ance3Classes.h5"
ECZEMA_MODEL = r'D:\Projects\Python Devolpment\Data field\python\data scintict\medicals-models\_models\eczemaH5v2.h5'
HEALTHY_SKIN_MODEL = r'D:\Projects\Python Devolpment\Data field\python\data scintict\medicals-models\_models\HealthySkin.h5'
ROSACEA_MODEL = r"D:\Projects\Python Devolpment\Data field\python\data scintict\medicals-models\_models\rosacea200.h5"
PSORIASIS_MODEL = r'D:\Projects\Python Devolpment\Data field\python\data scintict\medicals-models\_models\psoriasis.h5'
psoriasis_rosacea_MODEL = r'D:\Projects\Python Devolpment\Data field\python\data scintict\medicals-models\_models\psoriasis_rosacea.h5'
MODELS = {
    'rosacea': {
        'path': ROSACEA_MODEL,
        'type': 'h5',
        'classes': ['rosacea']
    },
    'acne': {
        'path': ACNE_MODEL,
        'type': 'h5',
        'classes': ['normal skin', 'skin acne', 'skin psoriasis']
    },
    'eczemaH5': {
        'path': ECZEMA_MODEL,
        'type': 'h5',
        'classes': ['Asteatotic Eczema', 'Chronic Eczema', 'Hand Eczema', 'Nummular Eczema', 'Subacute Eczema']
    },
    'HealthySkin': {
        'path': HEALTHY_SKIN_MODEL,
        'type': 'h5',
        'classes': ['Healthy-skin', 'Measles', 'Rubella']
    },
    'psoriasis': {
        'path': PSORIASIS_MODEL,
        'type': 'h5',
        'classes': ['Healthy-skin', 'Measles', 'Rubella']
    },
    'psoriasis_rosacea_MODEL': {
        'path': psoriasis_rosacea_MODEL,
        'type': 'h5',
        'classes': ['normal', 'psoriasis', 'rosacea']
    }
}

In [13]:

def load_trained_model(model_dir):
  '''
  this function take the model dir load it then from it know the input and the output shape
  Args
    model_dir => path of the model
  Return
    model, input_shape,classes in list(take len of it)

  '''

  model = tf.keras.models.load_model(model_dir)
  # return model
  # new edit
  input_shape = list(model.layers[0].input_shape[0])
  output_len = list(range(0, model.layers[-1].output.shape[1]))
  input_shape=input_shape[1:-1 ]
  return model, input_shape[::-1], output_len # input_shape[1:-1]

def classifier(img,model,shape)-> int:
    ''''''
    if img is None:
        raise ValueError("Image should not be None")

    # img=cv2.imread(img)
    
    img=cv2.resize(img,dsize=shape)
    # img_rgb=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
   
    img=np.expand_dims(img,axis=0)
    pred=model.predict(img,verbose=0)
    idx=np.argmax(pred)
    print('model output: ',pred)

    # values={k:i for i,k in train_batches.class_indices.items()}
    # print(values[idx])
    return idx,pred




def process_model(model,img):

    # if MODELS[model]['type'] == 'yolo':
    #     return None
    #     conf, c = yolo_prediction(img, MODELS[model]['path'])
    #     return model, {'conf': conf, 'classes': c}
    if MODELS[model]['type'] == 'h5':
        
        model_obj, shape, _ = load_trained_model(MODELS[model]['path'])
        print('model: ', model)
        idx, pred = classifier(img, model_obj, shape)
        print('pred:', pred)
        print('idx:', idx)
        print('__'*50)
       
        return model, {'conf': pred, 
                       'classes': MODELS[model]['classes'],
                       'class_int': int(idx)}



def predict_class(img): 

    results = {}
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_model = {executor.submit(process_model, model,img): model for model in MODELS}
        for future in concurrent.futures.as_completed(future_to_model):
            model = future_to_model[future]
            try:
                model_name, result = future.result()
                results[model_name] = result
            except Exception as exc:
                print(f'{model} generated an exception: {exc}')
    print('__'*50)
 
    return results
    

    # Find the model with the highest confidence
    # best_model = max(results, key=results.get)
    # return f'{best_model} found with confidence {results[best_model]}'


In [14]:
import matplotlib.pyplot as plt
import concurrent.futures

In [26]:
img = plt.imread('../../test_img/12.jpg')
x=predict_class(img)



model:  psoriasis_rosacea_MODEL
model:  eczemaH5
model:  rosacea
model output:  [[    0.99989  2.4762e-05  8.7852e-05]]
pred: [[    0.99989  2.4762e-05  8.7852e-05]]
idx: 0
____________________________________________________________________________________________________
model:  HealthySkin
model output:  [[ 0.00042066     0.50015  0.00071673     0.11345     0.38526]]
pred: [[ 0.00042066     0.50015  0.00071673     0.11345     0.38526]]
idx: 1
____________________________________________________________________________________________________
model output:  [[          1]]
pred: [[          1]]
idx: 0
____________________________________________________________________________________________________
model:  acne
model output:  [[    0.99997  3.1582e-05  6.6457e-09]]
pred: [[    0.99997  3.1582e-05  6.6457e-09]]
idx: 0
____________________________________________________________________________________________________
model:  psoriasis
model output:  [[  0.0039802   5.161e-06     0.9

In [23]:
def get_best_model(results):
    best_model = None
    best_confidence = -1
    best_class = None

    for model, result in results.items():
        max_acc = result['conf'].max()
        if max_acc > best_confidence and max_acc != 1:
            best_model = model
            best_confidence = max_acc
            best_class = result['classes'][result['conf'].argmax()]

    return best_model, best_class


best_model, best_class = get_best_model(x)
print(f'Best model: {best_model}, Class: {best_class}')

get_best_model(x)

Best model: psoriasis, Class: Healthy-skin


('psoriasis', 'Healthy-skin')

In [27]:
import pprint
pprint.pprint(x)



{'HealthySkin': {'class_int': 0,
                 'classes': ['Healthy-skin', 'Measles', 'Rubella'],
                 'conf': array([[    0.99997,  3.1582e-05,  6.6457e-09]], dtype=float32)},
 'acne': {'class_int': 2,
          'classes': ['normal skin', 'skin acne', 'skin psoriasis'],
          'conf': array([[  0.0039802,   5.161e-06,     0.99601]], dtype=float32)},
 'eczemaH5': {'class_int': 1,
              'classes': ['Asteatotic Eczema',
                          'Chronic Eczema',
                          'Hand Eczema',
                          'Nummular Eczema',
                          'Subacute Eczema'],
              'conf': array([[ 0.00042066,     0.50015,  0.00071673,     0.11345,     0.38526]], dtype=float32)},
 'psoriasis': {'class_int': 0,
               'classes': ['Healthy-skin', 'Measles', 'Rubella'],
               'conf': array([[          1,  3.1155e-06]], dtype=float32)},
 'psoriasis_rosacea_MODEL': {'class_int': 0,
                             'classes': ['ps

In [31]:
a=sorted(['psoriasis', 'rosacea', 'normal'])
a

['normal', 'psoriasis', 'rosacea']

In [17]:

x['psoriasis']

{'conf': array([[          1,  3.1155e-06]], dtype=float32),
 'classes': ['Healthy-skin', 'Measles', 'Rubella'],
 'class_int': 0}

In [19]:
for model in x:
    print(x[model])

{'conf': array([[    0.99997,  3.1582e-05,  6.6457e-09]], dtype=float32), 'classes': ['Healthy-skin', 'Measles', 'Rubella'], 'class_int': 0}
{'conf': array([[          1]], dtype=float32), 'classes': ['rosacea'], 'class_int': 0}
{'conf': array([[          1,  3.1155e-06]], dtype=float32), 'classes': ['Healthy-skin', 'Measles', 'Rubella'], 'class_int': 0}
{'conf': array([[  0.0039802,   5.161e-06,     0.99601]], dtype=float32), 'classes': ['normal skin', 'skin acne', 'skin psoriasis'], 'class_int': 2}
{'conf': array([[ 0.00042066,     0.50015,  0.00071673,     0.11345,     0.38526]], dtype=float32), 'classes': ['Asteatotic Eczema', 'Chronic Eczema', 'Hand Eczema', 'Nummular Eczema', 'Subacute Eczema'], 'class_int': 1}
